In [52]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
import typing_extensions
import psycopg
import pandas as pd
from IPython.display import clear_output
import re
import yfinance as yf
from datetime import date
import os

In [2]:
## connection block for pulling data from our server
conn_params = {
    "dbname": "silverbugs_db",
    "user": "elias_m",
    "password": "KSefPQeAhZJZM2x7jTdDHFT8i2gwGcnC",
    "host": "dpg-d1rfk1emcj7s73e3689g-a.oregon-postgres.render.com",
    "port": 5432
}

In [3]:
with psycopg.connect(**conn_params) as conn:
    df = pd.read_sql("SELECT id, title, selftext, created_utc, label FROM post_data;", conn)

/var/folders/0n/7phqtgwx3t3fvjqxcbzqtft80000gn/T/ipykernel_96150/328418087.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT id, title, selftext, created_utc, label FROM post_data;", conn)


In [4]:
df['silver_label'] = None
df['title+text'] = df['title'].astype('str') + df['selftext'].astype('str')
df

,id,title,selftext,created_utc,label,silver_label,title+text
0,1m5o61b,[WTS] 🔥THE BULLION DEAL YOU'VE BEEN WAITING FO...,Don't ask me why I'm doing this. Maybe I'm stu...,2025-07-21 16:50:51,None,None,[WTS] 🔥THE BULLION DEAL YOU'VE BEEN WAITING FO...
1,1m5o0ex,[WTS] - SILVER - 10oz Stackers - GOLD - 1/10 P...,PROOF: [https://imgur.com/a/gkqanQD](https://i...,2025-07-21 16:45:11,None,None,[WTS] - SILVER - 10oz Stackers - GOLD - 1/10 P...
2,1m5nupp,[wts] $200 of 90% Silver Junk 25x face value,"Looking for a whale to buy all. $25 of dimes,...",2025-07-21 16:39:26,None,None,[wts] $200 of 90% Silver Junk 25x face valueLo...
3,1m5nnh4,"[WTS] 14 silver rounds. ASE’s, Maples and a di...",Proof: https://imgur.com/a/jFfT9N3\n\nI have t...,2025-07-21 16:31:46,None,None,"[WTS] 14 silver rounds. ASE’s, Maples and a di..."
4,1m5niko,[WTS] 90% quarters and halves BELOW MELT,Proof : https://imgur.com/a/oy50U3U\n\nKitco s...,2025-07-21 16:26:35,None,None,[WTS] 90% quarters and halves BELOW MELTProof ...
...,...,...,...,...,...,...,...
1647,1m1ef22,"[WTS] [WTT] Platinum Kangaroo, Fractional gold...",I do not give out my password! My 2FA is enabl...,2025-07-16 14:43:24,WTS,None,"[WTS] [WTT] Platinum Kangaroo, Fractional gold..."
1648,1lz7ut2,[WTS] Rolls n' rolls of Rosie's 90%,[Proof](https://imgur.com/a/07-13-25-eNeCT9J)\...,2025-07-14 00:01:56,WTS,None,[WTS] Rolls n' rolls of Rosie's 90%[Proof](htt...
1649,1lv7cew,[WTS] Tuesday Gold & Silver 7-8-25,**Proof:** [Tuesday Gold & Silver 7-8-25](http...,2025-07-09 02:22:33,WTS,None,[WTS] Tuesday Gold & Silver 7-8-25**Proof:** [...
1650,1lv2ctv,[WTS] Gold & Silver (Premium & Melt),Proof https://imgur.com/a/ovQp07j\n\nLooking t...,2025-07-08 22:28:44,WTS,None,[WTS] Gold & Silver (Premium & Melt)Proof http...


In [5]:
df.sort_values(by = 'created_utc',ascending = False)

,id,title,selftext,created_utc,label,silver_label,title+text
136,1m6u2m4,"[WTS] Junk silver below spot, Toned and US typ...",[Proof](https://imgur.com/a/jgmVhPG)\n\nGood e...,2025-07-22 23:48:06,None,None,"[WTS] Junk silver below spot, Toned and US typ..."
141,1m6tzkl,"[wts] WTS – 90% Silver Halves & Quarters, Merc...",I started golfing a couple months back. In tha...,2025-07-22 23:44:26,None,None,"[wts] WTS – 90% Silver Halves & Quarters, Merc..."
142,1m6type,[WTB] 1987 Silver Eagle of Silver Maple Leaf,A friend of mine is turning 38 in a few weeks ...,2025-07-22 23:43:21,None,None,[WTB] 1987 Silver Eagle of Silver Maple LeafA ...
147,1m6twud,[wts] AU-AG-PT all at spot,Using kitco ask as spot. $10 priority shipping...,2025-07-22 23:40:59,None,None,[wts] AU-AG-PT all at spotUsing kitco ask as s...
148,1m6tjfz,[WTB] $10 BU Gold Indian,"Preferably ungraded, common date but i'll look...",2025-07-22 23:24:47,None,None,"[WTB] $10 BU Gold IndianPreferably ungraded, c..."
...,...,...,...,...,...,...,...
1168,1llc6c5,[WTS] FIVE BAGS LEFT! :O 90% Silver Walking Li...,This is a follow-up to my post from last week ...,2025-06-26 21:25:59,WTS,None,[WTS] FIVE BAGS LEFT! :O 90% Silver Walking Li...
1225,1llc1yw,[WTS] 90% Kennedy Halves - under spot whale lo...,[90% Kennedy Halves](https://imgur.com/a/3aLtt...,2025-06-26 21:20:59,WTS,None,[WTS] 90% Kennedy Halves - under spot whale lo...
1514,1llbr4t,[WTS] Incredible antique sterling flask and ji...,"Incredible, antique, sterling set up for grabs...",2025-06-26 21:08:50,WTS,None,[WTS] Incredible antique sterling flask and ji...
408,1llboln,[WTB] Palladium,"Looking to buy $600-1200 worth of palladium, i...",2025-06-26 21:06:02,WTB,None,[WTB] PalladiumLooking to buy $600-1200 worth ...


In [6]:
# uses regex to identify silver posts using relevant keywords

keywords = ['silver', 'ag', 'benjies', 'asw','90 percent', '90%', 'junk', 'silver eagles', 'constitutional', 'silver', 'liberties', 'walkers', 'mercs', 'mercuries', 'franklins', 'walkers', 'washingtons', 'washies', 'ASE', 'ASEs', 'roosevelts']
pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in keywords) + r')\b', flags=re.IGNORECASE)
pattern = re.compile(r'(' + '|'.join(keywords) + r')', re.IGNORECASE)

def label_silver_posts(text):
    if pattern.search(text):
        return 1
    else:
        return 0

df['silver_label'] = df['title+text'].apply(label_silver_posts)


In [7]:
# Regex code to recognize spot deals
spot_pattern = re.compile(r'\b(under|below|at|@)\s+spot\b', re.IGNORECASE)

# Apply to your DataFrame
df['spot_deal'] = df.apply(
    lambda row: 1 if row['silver_label'] == 1 and bool(spot_pattern.search(row['title+text'])) else 0,
    axis=1
)

In [8]:
# applies wts/wtb label
lowered_titles = df['title'].str.lower()

df.loc[lowered_titles.str.contains('wts'), 'label'] = 'WTS'
df.loc[lowered_titles.str.contains('wtb'), 'label'] = 'WTB'
df.label.value_counts()

label
WTS    1329
WTB     300
WTT      19
Name: count, dtype: int64

In [9]:
# drops exclusive WTT posts
wtt_index = df[df['label']=='WTT'].index
df = df.drop(wtt_index)

In [10]:
# creates dummy columns for wtb/wts columns
wtbs_cols = pd.get_dummies(df['label'],dtype = int)
df = pd.merge(wtbs_cols,df,left_index= True, right_index = True)

#changes date format to daily

df['created_utc'] = df['created_utc'].dt.date

#### create new df with daily values that we will use for ML

In [12]:
daily_values = df.groupby('created_utc').agg({
    'silver_label':'sum',
    'spot_deal': 'sum',
    'WTS': 'sum',
    'WTB': 'sum'}
    )
daily_values.reset_index(inplace = True)
daily_values['WTS_WTB_ratio'] = daily_values['WTS']/daily_values['WTB']
daily_values.rename(columns = {'created_utc':'date'},inplace = True)

In [13]:
daily_values

,date,silver_label,spot_deal,WTS,WTB,WTS_WTB_ratio
0,2025-06-26,21,6,21,3,7.000000
1,2025-06-27,43,6,41,11,3.727273
2,2025-06-28,49,7,44,9,4.888889
3,2025-06-29,45,8,38,12,3.166667
4,2025-06-30,50,18,51,10,5.100000
5,2025-07-01,48,11,48,16,3.000000
6,2025-07-02,52,8,54,5,10.800000
7,2025-07-03,58,7,51,13,3.923077
8,2025-07-04,43,8,37,9,4.111111
9,2025-07-05,41,10,42,10,4.200000


### Here we'll import the relevant silver and financial features we will sue for prediction

In [15]:
today = date.today()

In [16]:
tickers = ['SI=F', 'HG=F', 'SLV', 'PSLV', '^GSPC']


In [17]:
tickers = ['SI=F', 'HG=F', 'SLV', 'PSLV', '^GSPC'] 
silver_data = yf.download(tickers, start = '2025-06-26', end = today)

/var/folders/0n/7phqtgwx3t3fvjqxcbzqtft80000gn/T/ipykernel_96150/3883995620.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  silver_data = yf.download(tickers, start = '2025-06-26', end = today)
[*********************100%***********************]  5 of 5 completed


In [18]:
price_features = silver_data.reset_index()

In [19]:
price_features = price_features[['Date','Close','Volume']]
price_features.head()

Price        Date   Close                                           Volume  \
Ticker               HG=F   PSLV       SI=F        SLV        ^GSPC   HG=F   
0      2025-06-26  5.0655  12.45  36.585999  33.340000  6141.020020  17008   
1      2025-06-27  5.0685  12.15  36.036999  32.619999  6173.069824   4809   
2      2025-06-30  5.0300  12.24  35.852001  32.810001  6204.950195   1194   
3      2025-07-01  5.0480  12.24  36.082001  32.730000  6198.009766   1003   
4      2025-07-02  5.1490  12.43  36.425999  33.240002  6227.419922    792   

Price                                                
Ticker        PSLV   SI=F         SLV         ^GSPC  
0       18564000.0  43116  15255200.0  5.308140e+09  
1       28437900.0   7734  17596300.0  7.889350e+09  
2       18697300.0    191  10565300.0  5.782900e+09  
3       19566900.0     74  13436400.0  6.275310e+09  
4       24168600.0    512  13673500.0  5.645710e+09

In [20]:
price_features.columns = [f"{level_1}_{level_0}" for level_1,level_0 in price_features.columns]

In [21]:
price_features.head(1)

,Date_,Close_HG=F,Close_PSLV,Close_SI=F,Close_SLV,Close_^GSPC,Volume_HG=F,Volume_PSLV,Volume_SI=F,Volume_SLV,Volume_^GSPC
0,2025-06-26,5.0655,12.45,36.585999,33.34,6141.02002,17008,18564000.0,43116,15255200.0,5.308140e+09


In [22]:

price_cols = [col for col in price_features.columns if col.startswith("Close_")]

for col in price_cols:
    ticker = col.split("_")[1]  # Extract 'SLV', 'SI=F', etc.
    price_features[f"{ticker}_Daily_Return"] = price_features[col].pct_change()
    price_features[f"{ticker}_7_Day_Vol"] = price_features[col].pct_change().rolling(7).std()



/var/folders/0n/7phqtgwx3t3fvjqxcbzqtft80000gn/T/ipykernel_96150/898998326.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  price_features[f"{ticker}_Daily_Return"] = price_features[col].pct_change()
/var/folders/0n/7phqtgwx3t3fvjqxcbzqtft80000gn/T/ipykernel_96150/898998326.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  price_features[f"{ticker}_7_Day_Vol"] = price_features[col].pct_change().rolling(7).std()
/var/folders/0n/7phqtgwx3t3fvjqxcbzqtft80000gn/T/ipykernel_96150/898998326.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed i

In [48]:
price_features.rename(columns = {'Date_': 'Date',
                                'silver_label':'total posts'},inplace=True)

In [50]:
price_features_date_range = pd.date_range(start = '2025-06-26', end = today)
price_features = price_features.set_index('Date')

price_features


,Close_HG=F,Close_PSLV,Close_SI=F,Close_SLV,Close_^GSPC,Volume_HG=F,Volume_PSLV,Volume_SI=F,Volume_SLV,Volume_^GSPC,HG=F_Daily_Return,HG=F_7_Day_Vol,PSLV_Daily_Return,PSLV_7_Day_Vol,SI=F_Daily_Return,SI=F_7_Day_Vol,SLV_Daily_Return,SLV_7_Day_Vol,^GSPC_Daily_Return,^GSPC_7_Day_Vol
Date,,,,,,,,,,,,,,,,,,,,
2025-06-26,5.0655,12.45,36.585999,33.340000,6141.020020,17008.0,18564000.0,43116.0,15255200.0,5.308140e+09,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
2025-06-27,5.0685,12.15,36.036999,32.619999,6173.069824,4809.0,28437900.0,7734.0,17596300.0,7.889350e+09,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
2025-06-28,5.0685,12.15,36.036999,32.619999,6173.069824,4809.0,28437900.0,7734.0,17596300.0,7.889350e+09,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
2025-06-29,5.0685,12.15,36.036999,32.619999,6173.069824,4809.0,28437900.0,7734.0,17596300.0,7.889350e+09,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
2025-06-30,5.0300,12.24,35.852001,32.810001,6204.950195,1194.0,18697300.0,191.0,10565300.0,5.782900e+09,-0.007596,0.011698,0.007407,0.012559,-0.005134,0.009120,0.005825,0.011686,0.005164,0.005463
2025-07-01,5.0480,12.24,36.082001,32.730000,6198.009766,1003.0,19566900.0,74.0,13436400.0,6.275310e+09,0.003578,0.011698,0.000000,0.012559,0.006415,0.009120,-0.002438,0.011686,-0.001119,0.005463
2025-07-02,5.1490,12.43,36.425999,33.240002,6227.419922,792.0,24168600.0,512.0,13673500.0,5.645710e+09,0.020008,0.011698,0.015523,0.012559,0.009534,0.009120,0.015582,0.011686,0.004745,0.005463
2025-07-03,5.0970,12.54,36.784000,33.509998,6279.350098,0.0,25359100.0,0.0,10356200.0,3.378110e+09,-0.010099,0.011698,0.008850,0.012559,0.009828,0.009120,0.008123,0.011686,0.008339,0.005463
2025-07-04,5.0185,12.54,36.775002,33.509998,6279.350098,0.0,25359100.0,0.0,10356200.0,3.378110e+09,-0.015401,0.011698,0.000000,0.012559,-0.000245,0.009120,0.000000,0.011686,0.000000,0.005463


In [34]:
price_features = price_features.reindex(price_features_date_range)
price_features.reset_index(names = 'Date',inplace = True)

In [36]:

price_features.ffill(inplace= True)
price_features.bfill(inplace= True)


In [38]:
daily_values['Date'] = daily_values['date'].astype('<M8[ms]')

In [40]:
price_features

,Date,Close_HG=F,Close_PSLV,Close_SI=F,Close_SLV,Close_^GSPC,Volume_HG=F,Volume_PSLV,Volume_SI=F,Volume_SLV,...,HG=F_Daily_Return,HG=F_7_Day_Vol,PSLV_Daily_Return,PSLV_7_Day_Vol,SI=F_Daily_Return,SI=F_7_Day_Vol,SLV_Daily_Return,SLV_7_Day_Vol,^GSPC_Daily_Return,^GSPC_7_Day_Vol
0,2025-06-26,5.0655,12.45,36.585999,33.340000,6141.020020,17008.0,18564000.0,43116.0,15255200.0,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
1,2025-06-27,5.0685,12.15,36.036999,32.619999,6173.069824,4809.0,28437900.0,7734.0,17596300.0,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
2,2025-06-28,5.0685,12.15,36.036999,32.619999,6173.069824,4809.0,28437900.0,7734.0,17596300.0,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
3,2025-06-29,5.0685,12.15,36.036999,32.619999,6173.069824,4809.0,28437900.0,7734.0,17596300.0,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
4,2025-06-30,5.0300,12.24,35.852001,32.810001,6204.950195,1194.0,18697300.0,191.0,10565300.0,...,-0.007596,0.011698,0.007407,0.012559,-0.005134,0.009120,0.005825,0.011686,0.005164,0.005463
5,2025-07-01,5.0480,12.24,36.082001,32.730000,6198.009766,1003.0,19566900.0,74.0,13436400.0,...,0.003578,0.011698,0.000000,0.012559,0.006415,0.009120,-0.002438,0.011686,-0.001119,0.005463
6,2025-07-02,5.1490,12.43,36.425999,33.240002,6227.419922,792.0,24168600.0,512.0,13673500.0,...,0.020008,0.011698,0.015523,0.012559,0.009534,0.009120,0.015582,0.011686,0.004745,0.005463
7,2025-07-03,5.0970,12.54,36.784000,33.509998,6279.350098,0.0,25359100.0,0.0,10356200.0,...,-0.010099,0.011698,0.008850,0.012559,0.009828,0.009120,0.008123,0.011686,0.008339,0.005463
8,2025-07-04,5.0185,12.54,36.775002,33.509998,6279.350098,0.0,25359100.0,0.0,10356200.0,...,-0.015401,0.011698,0.000000,0.012559,-0.000245,0.009120,0.000000,0.011686,0.000000,0.005463
9,2025-07-05,5.0185,12.54,36.775002,33.509998,6279.350098,0.0,25359100.0,0.0,10356200.0,...,-0.015401,0.011698,0.000000,0.012559,-0.000245,0.009120,0.000000,0.011686,0.000000,0.005463


In [44]:
feature_set = pd.merge(daily_values, price_features, on = 'Date', how = 'left')

In [46]:
feature_set

,date,silver_label,spot_deal,WTS,WTB,WTS_WTB_ratio,Date,Close_HG=F,Close_PSLV,Close_SI=F,...,HG=F_Daily_Return,HG=F_7_Day_Vol,PSLV_Daily_Return,PSLV_7_Day_Vol,SI=F_Daily_Return,SI=F_7_Day_Vol,SLV_Daily_Return,SLV_7_Day_Vol,^GSPC_Daily_Return,^GSPC_7_Day_Vol
0,2025-06-26,21,6,21,3,7.000000,2025-06-26,5.0655,12.45,36.585999,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
1,2025-06-27,43,6,41,11,3.727273,2025-06-27,5.0685,12.15,36.036999,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
2,2025-06-28,49,7,44,9,4.888889,2025-06-28,5.0685,12.15,36.036999,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
3,2025-06-29,45,8,38,12,3.166667,2025-06-29,5.0685,12.15,36.036999,...,0.000592,0.011698,-0.024096,0.012559,-0.015006,0.009120,-0.021596,0.011686,0.005219,0.005463
4,2025-06-30,50,18,51,10,5.100000,2025-06-30,5.0300,12.24,35.852001,...,-0.007596,0.011698,0.007407,0.012559,-0.005134,0.009120,0.005825,0.011686,0.005164,0.005463
5,2025-07-01,48,11,48,16,3.000000,2025-07-01,5.0480,12.24,36.082001,...,0.003578,0.011698,0.000000,0.012559,0.006415,0.009120,-0.002438,0.011686,-0.001119,0.005463
6,2025-07-02,52,8,54,5,10.800000,2025-07-02,5.1490,12.43,36.425999,...,0.020008,0.011698,0.015523,0.012559,0.009534,0.009120,0.015582,0.011686,0.004745,0.005463
7,2025-07-03,58,7,51,13,3.923077,2025-07-03,5.0970,12.54,36.784000,...,-0.010099,0.011698,0.008850,0.012559,0.009828,0.009120,0.008123,0.011686,0.008339,0.005463
8,2025-07-04,43,8,37,9,4.111111,2025-07-04,5.0185,12.54,36.775002,...,-0.015401,0.011698,0.000000,0.012559,-0.000245,0.009120,0.000000,0.011686,0.000000,0.005463
9,2025-07-05,41,10,42,10,4.200000,2025-07-05,5.0185,12.54,36.775002,...,-0.015401,0.011698,0.000000,0.012559,-0.000245,0.009120,0.000000,0.011686,0.000000,0.005463


In [64]:
path = '/Users/elias_m/Desktop/silverbugs/data.feature_set.csv'

In [66]:
feature_set.to_csv(path_or_buf = path,index= False)